In [2]:
import pandas as pd
import numpy as np
from datetime import date
import datetime as dt

In [3]:

#将数据分为3个数据集 利用滑窗法
#将2016年1月1日到4月13日的数据提取特征，利用4月14日的到5月14日的作为测试集
#将2月1日到5月14日的作为数据集提取特征，利用5月15日6月15日的作为测试集
#将3月15日到6月30日作为数据集提取特征，再测试7月1日到7月31日的数据

In [4]:
"""
1 提取用户特征
    距离
    用户的平均距离，用户的最小距离，用户的最大距离
    使用优惠券买的物品数，买的总数，收到的优惠券数
    使用优惠券买的/总共收到的优惠券
"""
#利用pandas读取csv个格式的数据,header=None表示原文件没有索引
#原文件中总共有1754884个记录,header=0，表明第0行代表列名
off_train = pd.read_csv('C:\data\O2O_tianchi\ccf_offline_stage1_train.csv',header=0)
# print(off_train)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [5]:
off_train.columns=['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_test = pd.read_csv("C:\data\O2O_tianchi\ccf_offline_stage1_test_revised.csv",header=0)
off_test.head()
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [6]:
off_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
user_id          int64
merchant_id      int64
coupon_id        float64
discount_rate    object
distance         float64
date_received    float64
date             float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


In [7]:
#线上数据
on_train = pd.read_csv("C:\data\O2O_tianchi\ccf_online_stage1_train.csv",header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
on_train.head()

,user_id,merchant_id,action,coupon_id,discount_rate,date_received,date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0
2,14336199,18907,0,NaN,NaN,NaN,20160618.0
3,14336199,18907,0,NaN,NaN,NaN,20160618.0
4,14336199,18907,0,NaN,NaN,NaN,20160618.0


In [8]:
#使数据集等于test集
dataset3 = off_test
dataset3.head()
# #数据集3的特征为 取 线上数据中领券和用券日期大于3月15日和小于6月30日的
off_train['date']=off_train['date'].fillna(0)
off_train['date']=off_train['date'].astype('int')
off_train.head()
feature3 = off_train[((off_train['date'] >= 20160315)&(off_train.date<=20160630))|((off_train['date']==0)&(off_train['date_received']>=20160315)&(off_train['date_received']<=20160630))]
feature3.head()
# type(off_train['date']Date
#提取数据集2的测试集
dataset2 = off_train[(off_train['date_received']>=20160515)&(off_train['date_received']<=20160615)]
dataset2.head()
#在2月1日到5月14日之间使用了券,只要领取时间在2月1日到5月14日之间,并包括没有数据中没有领取券的
feature2 = off_train[(off_train['date']>=20160201)&(off_train['date']<=20160514)|((off_train['date']==0)&(off_train['date_received']>=20160201)&(off_train['date_received']<=20160514))]
feature2.head()
#同理可得
dataset1 = off_train[(off_train['date_received']>=201604014)&(201604014<=20160514)]
feature1 = off_train[(off_train['date']>=20160101)&(off_train.date<=20160413)|((off_train['date']==0)&(off_train['date_received']>=20160101)&(off_train['date_received']<=20160413))]
feature1.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217
2,1439408,2632,8591.0,20:1,0.0,20160217.0,0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,0
8,2029232,3381,11951.0,200:20,1.0,20160129.0,0
16,2223968,3381,9776.0,10:5,2.0,20160129.0,0


In [9]:
"""
这里用到的特征有
    用户领取的所有优惠券数目
    ◦用户领取的特定优惠券数目
    ◦用户此次之后/前领取的所有优惠券数目
    ◦用户此次之后/前领取的特定优惠券数目
    ◦用户上/下一次领取的时间间隔
    ◦用户领取特定商家的优惠券数目
    ◦用户领取的不同商家数目
    ◦用户当天领取的优惠券数目
    ◦用户当天领取的特定优惠券数目
    ◦用户领取的所有优惠券种类数目
    ◦商家被领取的优惠券数目
    ◦商家被领取的特定优惠券数目
    ◦商家被多少不同用户领取的数目
    ◦商家发行的所有优惠券种类数目
这些特征都是在预测集中被提取的
"""
#对于测试集3，提取用户的Id
dataset3.head()
t = dataset3[['User_id']].copy()
t.head()
# #相当于给原有数据加上一列，这个月用户收取的所有优惠券数目，并初始化为1
t['this_month_user_receive_all_coupon_count']=1
t.head()

#将t按照用户id进行分组，然后统计所有用户收取的优惠券数目,并初始化一个索引值
t = t.groupby('User_id').agg('sum').reset_index()
t.head()
#提取数据集3的优惠券Id和用户Id
t1 = dataset3[['User_id','Coupon_id']].copy()
#提取这个月用户收到的相同的优惠券的数量
t1['this_month_user_receive_same_coupn_count'] = 1
t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

#提取数据集3的用户id，优惠券id以及优惠券接收的时间
t2 = dataset3[['User_id','Coupon_id','Date_received']].copy()
#将数据转换为str类型
t2['Date_received'] = t2['Date_received'].astype('str')
#如果出现相同的用户接收相同的优惠券在接收时间上用‘：’连接上第n次接受优惠券的时间
t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
# t2 = t2.groupby(['User_id','Coupon_id']).agg(lambda x:':'.join(x)).reset_index()
t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
t2.head(30)
# #将接收时间的一组按着':'分开，这样就可以计算接受了优惠券的数量,apply是合并
t2['receive_number'] = t2['Date_received'].apply(lambda s:len(s.split(':')))
t2 = t2[t2['receive_number'] > 1]
t2.head()

#最大接受的日期
t2['max_date_received'] = t2['Date_received'].apply(lambda s:max([int (d) for d in s.split(':')]))
#最小的接收日期
t2['min_date_received'] = t2['Date_received'].apply(lambda s:min([int (d) for d in s.split(':')]))
t2.head()
t2 = t2[['User_id','Coupon_id','max_date_received','min_date_received']]
t3 = dataset3[['User_id','Coupon_id','Date_received']]

#将两表融合只保留左表数据,这样得到的表，相当于保留了最近接收时间和最远接受时间
t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
#这个优惠券最近接受时间
t3['this_month_user_receive_same_coupon_lastone']= t3['max_date_received']-t3['Date_received'].astype(int)
#这个优惠券最远接受时间
t3['this_month_user_receive_same_coupon_firstone'] = t3['Date_received'].astype(int)-t3['min_date_received']

def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #表明这个优惠券只接受了一次
t3['this_month_user_receive_same_coupon_lastone'] = t3['this_month_user_receive_same_coupon_lastone'].apply(is_firstlastone)
t3['this_month_user_receive_same_coupon_firstone'] = t3['this_month_user_receive_same_coupon_firstone'].apply(is_firstlastone)
t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

#提取第四个特征,一个用户所接收到的所有优惠券的数量
t4 = dataset3[['User_id','Date_received']]
t4.loc['this_day_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()
#提取第五个特征,一个用户不同时间所接收到不同优惠券的数量
t5 = dataset3[['User_id','Coupon_id','Date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
#一个用户不同优惠券 的接受时间
t6 = dataset3[['User_id','Coupon_id','Date_received']]
t6.Date_received = t6.Date_received.astype('str')
t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
#重命名inplace代表深拷贝
t6.rename(columns={'Date_received':'dates'},inplace = True)

def get_day_gap_before(s):
    Date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        #将时间差转化为天数
        this_gap = (dt.date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))-dt.date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
def get_day_gap_after(s):
    Date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (dt.datetime(int(d[0:4]),int(d[4:6]),int(d[6:8]))-dt.datetime(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [10]:
t7 = dataset3[['User_id','Coupon_id','Date_received']]
#将t6和t7融合
t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
#注意这里所有的时间格式都已经是'str'格式
t7['date_received_date'] = t7.Date_received.astype('str')+'-'+t7.dates
#print(t7)
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]

In [11]:
#将所有特征融合在一张表中
other_feature3 = pd.merge(t1,t,on='User_id')
other_feature3 = pd.merge(other_feature3,t3,on=['User_id','Coupon_id'])
other_feature3 = pd.merge(other_feature3,t4,on=['User_id','Date_received'])
other_feature3 = pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
other_feature3 = pd.merge(other_feature3,t7,on=['User_id','Coupon_id','Date_received'])
other_feature3.to_csv('C:\data\other_feature3.csv',index=None)
#print(other_feature3)

In [12]:
#提取优惠券的相关特征

def calc_discount_rate(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])
def get_discount_man(s):
    s = str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[0])
def get_discount_jian(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return int(s[1])

def is_man_jian(s):
    s = str(s)
    s = s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1

In [13]:

#显示时间是第几周
dataset3['day_of_week'] = dataset3['Date_received'].astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
#显示时间是几月
dataset3['day_of_month'] = dataset3['Date_received'].astype('str').apply(lambda x:int(x[6:8]))
#显示时期和截止日之间的天数
dataset3['days_distance'] = dataset3['Date_received'].astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)
#显示满了多少钱后开始减
dataset3['discount_man'] = dataset3['Discount_rate'].apply(get_discount_man)
#显示满减的减少的钱
dataset3['discount_jian'] = dataset3['Discount_rate'].apply(get_discount_jian)
#返回优惠券是否是满减券
dataset3['is_man_jian'] = dataset3['Discount_rate'].apply(is_man_jian)
#显示打折力度
dataset3['Discount_rate'] = dataset3['Discount_rate'].apply(calc_discount_rate)
d = dataset3[['Coupon_id']]
d['coupon_count'] = 1
#显示每一种优惠券的数量
d = d.groupby('Coupon_id').agg('sum').reset_index()
dataset3 = pd.merge(dataset3,d,on='Coupon_id',how='left')
# print(dataset3)
dataset3.to_csv('C:\data\coupon3_feature.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
dataset2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
1,1439408,4663,11002.0,150:20,1.0,20160528.0,0
4,1439408,2632,8591.0,20:1,0.0,20160613.0,0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613
9,2029232,450,1532.0,30:5,0.0,20160530.0,0
10,2029232,6459,12737.0,20:1,0.0,20160519.0,0


In [15]:
#对于数据集2
dataset2['day_of_week'] = dataset2['date_received'].astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset2['day_of_month'] = dataset2['date_received'].astype('str').apply(lambda x:int(x[6:8]))
dataset2['days_distance'] = dataset2['date_received'].astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,5,14)).days)
dataset2['discount_man'] = dataset2['discount_rate'].apply(get_discount_man)
dataset2['discount_jian'] = dataset2['discount_rate'].apply(get_discount_jian)
dataset2['is_man_jian'] = dataset2['discount_rate'].apply(is_man_jian)
dataset2['discount_rate'] = dataset2['discount_rate'].apply(calc_discount_rate)
d = dataset2[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset2 = pd.merge(dataset2,d,on='coupon_id',how='left')
dataset2.to_csv('C:\data\coupon2_feature.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [16]:
#对于数据集1
dataset1['day_of_week'] = dataset1['date_received'].apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset1['day_of_month'] = dataset1['date_received'].apply(lambda x:int(x[6:8]))
dataset1['days_distance'] = dataset1['date_received'].apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,4,13)).days)
dataset1['discount_man'] = dataset1['discount_rate'].apply(get_discount_man)
dataset1['discount_jian'] = dataset1['discount_rate'].apply(get_discount_jian)
dataset1['is_man_jian'] = dataset1['discount_rate'].apply(is_man_jian)
dataset1['discount_rate'] = dataset1['discount_rate'].apply(calc_discount_rate)
d = dataset1[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset1 = pd.merge(dataset1,d,on='coupon_id',how='left')
dataset1.to_csv('C:\data\coupon1_feature.csv',index=None)

In [17]:
#提取商品的特征

#对于数据集3
merchant3 = feature3[['merchant_id','coupon_id','distance','date_received','date']]

t = merchant3[['merchant_id']]
#删除重复行数据
t.drop_duplicates(inplace=True)
merchant3.head()
#显示卖出的商品
t1 = merchant3[merchant3.date!=0][['merchant_id']]
t1['total_sales'] = 1
#显示每个商品的销售数量
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
#显示使用了优惠券消费的商品，正样本
t2 = merchant3[(merchant3.date!=0)&(merchant3.coupon_id!=0)][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
#显示了商品的优惠券的总数量
t3 = merchant3[merchant3.coupon_id != 0][['merchant_id']]
t3 ['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()

#显示商品销量和距离的关系
t4 = merchant3[(merchant3.date != 0)&(merchant3.coupon_id != 0)][['merchant_id','distance']]
#把数据中的null值全部替换为-1
t4.replace('null',-1,inplace=True)
t4.distance=t4.distance.fillna(1)
t4.distance = t4.distance.astype('int')
#再把数据中的-1全部替换为NaN
t4.replace(-1,np.nan,inplace=True)
#返回用户离商品的距离最小值
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
#返回用户离商品的距离最大值
t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace = True)
#print(t6)
#返回距离的平均值
t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns = {'distance':'merchant_mean_distance'},inplace= True)
#返回距离的中位值
t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace = True)
merchant3_feature = pd.merge(t,t1,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t2,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t3,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t5,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t6,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t7,on='merchant_id',how='left')
#print(merchant3_feature)
merchant3_feature = pd.merge(merchant3_feature,t8,on='merchant_id',how='left')
#print(merchant3_feature)

In [19]:
#将数据中的NaN用0来替换
merchant3_feature.sales_use_coupon = merchant3_feature.sales_use_coupon.replace(np.nan,0)
#即优惠券的使用率
merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon.astype('float')/merchant3_feature.total_coupon
#即卖出商品中使用优惠券的占比
merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_sales
#将数据中的NaN用0来替换
merchant3_feature.total_coupon = merchant3_feature.total_coupon.replace(np.nan,0)
merchant3_feature.to_csv('C:\data\merchant3_feature.csv',index=None)

In [20]:
#4.user_merchant:
      #times_user_buy_merchant_before. 

all_user_merchant = feature3[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)
#只保留销售了商品的商户id
t = feature3[['user_id','merchant_id','date']]
t = t[t.date!=0][['user_id','merchant_id']]
#用户一共买了这家商户的多少商品
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)

t1 = feature3[['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id!=0][['user_id','merchant_id']]
#用户一共收到一个商户的多少优惠券
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)


t2 = feature3[['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!=0)&(t2.date_received!=0)][['user_id','merchant_id']]

#用户在一家商户中使用优惠券购买的商品的数目
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace = True)

#用户在一家商家的所有记录总数
t3 = feature3[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace = True)

t4 = feature3[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date!=0)&(t4.coupon_id==0)][['user_id','merchant_id']]



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
#用户没有使用优惠券购买的商品的数目
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace = True)

user_merchant3 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t1,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t2,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t3,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t4,on=['user_id','merchant_id'],how='left')
#都是针对一家商户和一个用户
user_merchant3.user_merchant_buy_use_coupon = user_merchant3.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant3.user_merchant_buy_common = user_merchant3.user_merchant_buy_common.replace(np.nan,0)
#y优惠券的转换率，用户使用了的优惠券/一共收到的优惠券
user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')
#用户使用优惠券的概率，在一家商户使用优惠券购买的商品/在一家商户购买商品的总数
user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
#用户在商户消费的概率 用户在商户购买的总数/在一家商户浏览的总次数
user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float') / user_merchant3.user_merchant_any.astype('float')
#用户在一家商户不适用优惠券购买的概率 普通购买的商品数/购买商品的总数
user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3.to_csv('C:/data/user_merchant3.csv',index=None)

In [22]:
#对于数据集2
all_user_merchant = feature2[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)
t = feature2[['user_id','merchant_id','date']]
t = t[t.date!=0][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace = True)
t1 = feature2[['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id != 0][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)
t2 = feature2[['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!=0)&(t2.date_received !=0)][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace= True)
t3 = feature2[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
t4 = feature2[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date !=0)&(t4.coupon_id == 0)][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)

user_merchant2 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t1,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t2,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t3,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t4,on=['user_id','merchant_id'],how='left')

user_merchant2.user_merchant_buy_use_coupon = user_merchant2.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant2.user_merchant_buy_common = user_merchant2.user_merchant_buy_common.replace(np.nan,0)

user_merchant2['user_merchant_coupon_transfer_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float') / user_merchant2.user_merchant_received.astype('float')
user_merchant2['user_merchant_coupon_buy_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float') / user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2['user_merchant_rate'] = user_merchant2.user_merchant_buy_total.astype('float') / user_merchant2.user_merchant_any.astype('float')
user_merchant2['user_merchant_common_buy_rate'] = user_merchant2.user_merchant_buy_common.astype('float') / user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2.to_csv('C:\data\\user_merchant2.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
#for dataset2
all_user_merchant = feature1[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)

t = feature1[['user_id','merchant_id','date']]
t = t[t.date!=0][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)

t1 = feature1[['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id!=0][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
t2 = feature1[['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!=0)&(t2.date_received!=0)][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)

In [29]:
t3 = feature1[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
t4 = feature1[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date!=0)&(t4.coupon_id==0)][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)

In [31]:
user_merchant1 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t1,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t2,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t3,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t4,on=['user_id','merchant_id'],how='left')

In [33]:
user_merchant1.user_merchant_buy_use_coupon = user_merchant1.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant1.user_merchant_buy_common = user_merchant1.user_merchant_buy_common.replace(np.nan,0)
user_merchant1['user_merchant_coupon_transfer_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float') / user_merchant1.user_merchant_received.astype('float')
user_merchant1['user_merchant_coupon_buy_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float') / user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1['user_merchant_rate'] = user_merchant1.user_merchant_buy_total.astype('float') / user_merchant1.user_merchant_any.astype('float')
user_merchant1['user_merchant_common_buy_rate'] = user_merchant1.user_merchant_buy_common.astype('float') / user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1.to_csv('C:\data\\user_merchant1.csv',index=None)

In [77]:
#构建衍生特征
############# user related feature   #############
"""
3.user related: 
      count_merchant. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      buy_use_coupon/buy_total
      user_date_datereceived_gap
      

"""

def get_user_date_datereceived_gap(s):
    s = s.split(':')
    return (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days

#for dataset3
user3 = feature3[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]

t = user3[['user_id']]
t.drop_duplicates(inplace=True)

t = user3[['user_id']]
t.drop_duplicates(inplace=True)

t1 = user3[user3.date!=0][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)

t2 = user3[(user3.date!=0)&(user3.coupon_id!=0)][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.distance = t2.distance.fillna(0)
t2.distance = t2.distance.astype('int')
t2.replace(-1,np.nan,inplace=True)
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_distance'},inplace=True)


t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)

t7 = user3[(user3.date!=0)&(user3.coupon_id!=0)][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()


t8 = user3[user3.date!=0][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()

t9 = user3[user3.coupon_id!=0][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()

t10 = user3[(user3.date_received!=0)&(user3.date!=0)][['user_id','date_received','date']]
t10.date=t10.date.astype(str)
t10.date_received=t10.date.astype(str)
t10['user_date_datereceived_gap']=t10.apply(lambda x: x['date']+ ':'+ x['date'],axis=1)
t10['user_date_datereceived_gap'].head()
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)

t10 = t10[['user_id','user_date_datereceived_gap']]

t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)

user3_feature = pd.merge(t,t1,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='user_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.coupon_received.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.to_csv('C:/data/user3_feature.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [79]:
#for dataset2
user2 = feature2[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]

t = user2[['user_id']]
t.drop_duplicates(inplace=True)

t1 = user2[user2.date!=0][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)

t2 = user2[(user2.date!=0)&(user2.coupon_id!=0)][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.distance = t2.distance.astype('int')
t2.replace(-1,np.nan,inplace=True)
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_distance'},inplace=True)

t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)

t7 = user2[(user2.date!='null')&(user2.coupon_id!='null')][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()

t8 = user2[user2.date!='null'][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()

t9 = user2[user2.coupon_id!='null'][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()

t10 = user2[(user2.date_received!='null')&(user2.date!='null')][['user_id','date_received','date']]
t10['user_date_datereceived_gap'] = t10.date + ':' + t10.date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]

t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)

user2_feature = pd.merge(t,t1,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t3,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t4,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t5,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t6,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t7,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t8,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t9,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t11,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t12,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t13,on='user_id',how='left')
user2_feature.count_merchant = user2_feature.count_merchant.replace(np.nan,0)
user2_feature.buy_use_coupon = user2_feature.buy_use_coupon.replace(np.nan,0)
user2_feature['buy_use_coupon_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.buy_total.astype('float')
user2_feature['user_coupon_transfer_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.coupon_received.astype('float')
user2_feature.buy_total = user2_feature.buy_total.replace(np.nan,0)
user2_feature.coupon_received = user2_feature.coupon_received.replace(np.nan,0)
user2_feature.to_csv('C:/data/user2_feature.csv',index=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


ValueError: Cannot convert non-finite values (NA or inf) to integer